In [2]:
!pip install simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import sklearn
from sklearn.metrics import classification_report
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

2021-08-12 16:39:41.673440: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [ ]:

# Preparing train data
train_df = pd.read_csv("/content/drive/MyDrive/Master Thesis/train.csv")
train_df["trust"] = train_df["trust"].replace(["untrusted","trusted"],[0,1])
train_df = train_df[["article", "trust"]]
# Preparing eval data
eval_df = pd.read_csv("/content/drive/MyDrive/Master Thesis/test.csv")
eval_df["trust"] = eval_df["trust"].replace(["untrusted","trusted"],[0,1])
eval_df = eval_df[["article", "trust"]]

# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)

# Create a ClassificationModel
model = ClassificationModel(
    "xlmroberta","xlm-roberta-base", args=model_args
)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df, classification_report=sklearn.metrics.classification_report)

In [10]:
sent_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/annotation_set_sent.csv")
sent_dataset["sentence"] = sent_dataset["article"].str.split("\n")
sent_dataset = sent_dataset.explode("sentence")
sent_dataset["sent_score"] = sent_dataset["article"].str.extract(r"\[([0-9])\]")
sent_dataset["sent_score"] = sent_dataset["sent_score"].astype(int)
train = sent_dataset[0:1200]
dev = sent_dataset[1200:1350]
test = sent_dataset[1350:1500]
train = train[["sentence", "sent_score"]]
dev = dev[["sentence", "sent_score"]]
test = test[["sentence", "sent_score"]]

model_args = ClassificationArgs(num_train_epochs=1)

model = ClassificationModel(
    "xlmroberta","xlm-roberta-base", num_labels=6, args=model_args
)

model.train_model(train)
result, model_outputs, wrong_predictions = model.eval_model(test, classification_report=sklearn.metrics.classification_report)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/150 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:942: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm
INFO:simpletransformers.classification.classification_model: Training of xlmroberta model complete. Saved to outputs/.
/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1402: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/150 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_xlmroberta_128_6_2


Running Evaluation:   0%|          | 0/19 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
INFO:simpletransformers.classification.classification_model:{'mcc': 0.2950623287309555, 'classification_report': '              precision    recall  f1-score   support\n\n         0.0       0.25      0.60      0.35        20\n         1.0       0.81      0.83      0.82       100\n         5.0       0.00      0.00      0.00        30\n\n    accuracy                           0.63       150\n   macro avg       0.35      0.48      0.39       150\nweighted avg       0.58      0.63      0.59       150\n', 'eval_loss': 0.9242534974687978}
